### Setup

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
wbdta = pd.read_excel('Data/World_Development_Indicators.xlsx', sheet_name='Data')
LCOEdta = pd.read_csv('Data/LCOE All - IEA.csv')
CoGdta = pd.read_excel('Data/levelized-capital-costs-of-electricity-generation-in-the-us-2027.xlsx', sheet_name='Data')
plants1 = pd.read_csv('Data/global_power_plant_database.csv')
matprice = pd.read_excel('Data/Primary_Commodity_Price.xlsx', sheet_name='Data')
matuse = pd.read_csv('Data/minerals-used-in-clean-energy.csv')


c:\Users\agrus\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### World Bank stuff

In [3]:
# get the list of columns that contain year values
year_cols = wbdta.columns[4:].tolist()

# create an empty list to store the pivoted dataframes
dfs_list = []

# loop over the unique values of Series Name
for series_name in wbdta['Series Name'].unique():
    
    # subset the dataframe for the current series
    holder = wbdta.loc[wbdta['Series Name']==series_name, ['Country Name', 'Country Code'] + year_cols]
    
    # pivot the dataframe to the desired format
    holder_pivot = holder.melt(id_vars = ['Country Name','Country Code'], var_name = 'Year', value_name = series_name)

    # set the name of the new dataframe
    holder_pivot.columns.name = None

    if series_name == "Energy intensity level of primary energy (MJ/$2017 PPP GDP)":
        dfs_list.append(holder_pivot['Country Name'])
        dfs_list.append(holder_pivot['Country Code'])
        dfs_list.append(holder_pivot['Year'])
        dfs_list.append(holder_pivot[series_name])
    else:
        dfs_list.append(holder_pivot[series_name])


# merge all dataframes into a single dataframe
merged_df = pd.concat(dfs_list, axis=1)

In [4]:
hold_bin = []
yval = []
#reformat the year values
for x in merged_df['Year']:
    holder = x.split(' ')
    yval.append(int(holder[0]))
    #change the year to datetime
    holder = pd.to_datetime(holder[0], format='%Y')

    hold_bin.append(holder)

merged_df['Yearint'] = yval
merged_df['Year'] = hold_bin


#fix na system
merged_df.replace('..', np.nan, inplace=True)


merged_df.head()

,Country Name,Country Code,Year,Energy intensity level of primary energy (MJ/$2017 PPP GDP),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Access to electricity (% of population),Electricity production from coal sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Electricity production from hydroelectric sources (% of total),...,Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Value lost due to electrical outages (% of sales for affected firms),Adjusted savings: energy depletion (% of GNI),Oil rents (% of GDP),Fuel imports (% of merchandise imports),Fuel exports (% of merchandise exports),Natural gas rents (% of GDP),Coal rents (% of GDP),Yearint
0,Afghanistan,AFG,1971-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.011334,0.029425,5.000146,14.553298,0.000000,NaN,1971
1,Albania,ALB,1971-01-01,NaN,NaN,NaN,NaN,0.000000,42.857143,57.142857,...,532.028432,785.161526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1971
2,Algeria,DZA,1971-01-01,NaN,NaN,NaN,NaN,0.000000,85.195155,14.804845,...,141.102889,245.527602,NaN,5.179009,8.051916,3.487314,74.852037,0.015897,0.002527,1971
3,Angola,AGO,1971-01-01,NaN,NaN,NaN,NaN,0.000000,18.463612,81.536388,...,90.010618,623.396059,NaN,NaN,NaN,3.008041,19.947138,NaN,NaN,1971
4,Argentina,ARG,1971-01-01,NaN,NaN,NaN,NaN,3.085845,93.222994,6.535726,...,875.765516,1389.076933,NaN,0.306210,0.394583,6.682968,0.488016,0.021561,0.000000,1971


In [5]:
#drop worthless stuff
droplist = ['Oil rents (% of GDP)','Natural gas rents (% of GDP)', 'Coal rents (% of GDP)',
            'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
            'Access to electricity, rural (% of rural population)', 'Access to electricity, urban (% of urban population)',
            'Access to electricity (% of population)', 'Value lost due to electrical outages (% of sales for affected firms)',
            'Fuel exports (% of merchandise exports)'
            ]

merged_df = merged_df.drop(droplist, axis=1)
merged_df.shape

(7800, 26)

In [6]:
aggtable = merged_df[merged_df['Yearint'].astype(int) > 2018]

merged_df = merged_df.drop('Yearint', axis=1)
aggtable = aggtable.drop('Yearint', axis=1)

aggtable = aggtable.groupby('Country Name').mean().reset_index()

In [7]:
# save the dataframe to a CSV file
merged_df.to_csv('Final/CleanData/WB_clean.csv', index=False)

### LCOE

In [8]:
print(LCOEdta.shape)
LCOEdta.head(3)

(237, 17)


,Country,Plant category,Plant type,Construction costs (USD/MWh),Refurbishment costs (USD/MWh),Decommissioning costs (USD/MWh),Total capital costs (USD/MWh),Operations & maintenance costs (USD/MWh),Fuel costs (th) (USD/MWh),Fuel costs (el) (USD/MWh),Carbon price (USD/ton),Carbon costs (USD/MWh),Heat price (USD/MWh),CHP heat revenues (USD/MWh),Discount rate,LCOE (USD/MWh),Energy type
0,Japan,Nuclear,ALWR (1152 MW),46.87,0.0,0.05,46.92,25.84,0.00,13.92,30,0.0,37.06,0.0,0.07,86.67,Nuclear
1,"Korea, Republic of",Nuclear,ALWR (1377 MW),25.51,0.0,0.03,25.54,18.44,0.00,9.33,30,0.0,37.06,0.0,0.07,53.30,Nuclear
2,Italy,Biomass,Biomass (0.42 MW),70.23,0.0,0.19,70.42,59.00,15.62,159.42,30,0.0,37.06,0.0,0.07,288.84,Biomass


In [9]:
LCOEdta.columns
#th = thermal. el = electrical

Index(['Country', 'Plant category', 'Plant type',
       'Construction costs (USD/MWh)', 'Refurbishment costs (USD/MWh)',
       'Decommissioning costs (USD/MWh)', 'Total capital costs (USD/MWh)',
       'Operations & maintenance costs (USD/MWh)', 'Fuel costs (th) (USD/MWh)',
       'Fuel costs (el) (USD/MWh)', 'Carbon price (USD/ton)',
       'Carbon costs (USD/MWh)', 'Heat price (USD/MWh)',
       'CHP heat revenues (USD/MWh)', 'Discount rate', 'LCOE (USD/MWh)',
       'Energy type'],
      dtype='object')

In [10]:
#combine fuel cost
LCOEdta['Total Fuel cost (USD/MWh)'] = LCOEdta['Fuel costs (el) (USD/MWh)'] + LCOEdta['Fuel costs (th) (USD/MWh)']

droplist2 = ['CHP heat revenues (USD/MWh)','Discount rate','Fuel costs (el) (USD/MWh)',
             'LCOE (USD/MWh)','Carbon costs (USD/MWh)','Fuel costs (th) (USD/MWh)',
             'Heat price (USD/MWh)','Carbon price (USD/ton)',
             ]

#drop useless stuff
LCOEdta = LCOEdta.drop(droplist2, axis=1)

print(LCOEdta.shape)
LCOEdta.head(3)

(237, 10)


,Country,Plant category,Plant type,Construction costs (USD/MWh),Refurbishment costs (USD/MWh),Decommissioning costs (USD/MWh),Total capital costs (USD/MWh),Operations & maintenance costs (USD/MWh),Energy type,Total Fuel cost (USD/MWh)
0,Japan,Nuclear,ALWR (1152 MW),46.87,0.0,0.05,46.92,25.84,Nuclear,13.92
1,"Korea, Republic of",Nuclear,ALWR (1377 MW),25.51,0.0,0.03,25.54,18.44,Nuclear,9.33
2,Italy,Biomass,Biomass (0.42 MW),70.23,0.0,0.19,70.42,59.00,Biomass,175.04


In [11]:
holdr = []
for n in LCOEdta['Plant type']:
    n = n.split(' (')
    n = n[0]
    holdr.append(n)

LCOEdta['energy_type'] = holdr

In [12]:
aggtable = pd.merge(LCOEdta,aggtable,left_on='Country', right_on='Country Name',how='left')
aggtable.drop('Country Name', axis=1)
aggtable.head()

,Country,Plant category,Plant type,Construction costs (USD/MWh),Refurbishment costs (USD/MWh),Decommissioning costs (USD/MWh),Total capital costs (USD/MWh),Operations & maintenance costs (USD/MWh),Energy type,Total Fuel cost (USD/MWh),...,GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent),"Energy imports, net (% of energy use)",Alternative and nuclear energy (% of total energy use),Fossil fuel energy consumption (% of total),"Energy use (kg of oil equivalent) per $1,000 GDP (constant 2017 PPP)",Combustible renewables and waste (% of total energy),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Adjusted savings: energy depletion (% of GNI),Fuel imports (% of merchandise imports)
0,Japan,Nuclear,ALWR (1152 MW),46.87,0.0,0.05,46.92,25.84,Nuclear,13.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003992,19.091167
1,"Korea, Republic of",Nuclear,ALWR (1377 MW),25.51,0.0,0.03,25.54,18.44,Nuclear,9.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001910,21.908541
2,Italy,Biomass,Biomass (0.42 MW),70.23,0.0,0.19,70.42,59.00,Biomass,175.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054715,10.429541
3,Italy,Biomass,Biomass (0.45 MW),48.93,0.0,0.13,49.07,26.89,Biomass,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054715,10.429541
4,Brazil,Biomass,Biomass (25.0 MW),34.68,0.0,0.09,34.78,18.75,Biomass,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.632956,11.979674


In [13]:
# save the dataframe to a CSV file
#aggtable.to_csv('Final/CleanData/LCOE.csv', index=False)
LCOEdta.to_csv('Final/CleanData/LCOE.csv', index=False)

### Statista

In [14]:
print(CoGdta.shape)
CoGdta.head()

(12, 2)


,Energy Source,Levelized Cost of Generation($ per megawatt hour)
0,Offshore wind,103.77
1,Battery storage,64.03
2,Advanced nuclear,60.71
3,Combustion turbine,53.78
4,Ultra-supercritical coal,52.11


In [15]:
CoGdta['Energy Source']

0                Offshore wind
1              Battery storage
2             Advanced nuclear
3           Combustion turbine
4     Ultra-supercritical coal
5                        Hydro
6                      Biomass
7               Solar (hybrid)
8                 Onshore wind
9           Solar (standalone)
10                  Geothermal
11              Combined cycle
Name: Energy Source, dtype: object

In [16]:
# save the dataframe to a CSV file
CoGdta.to_csv('Final/CleanData/capital-costs-of-electricity.csv', index=False)

### Class Dataset

In [17]:
print(plants1.shape)
plants1.head(3)

(34936, 36)


,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.119,Hydro,NaN,NaN,...,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.795,Solar,NaN,NaN,...,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.792,Solar,NaN,NaN,...,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE


In [18]:
plants1.columns

Index(['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018',
       'generation_gwh_2019', 'generation_data_source',
       'estimated_generation_gwh_2013', 'estimated_generation_gwh_2014',
       'estimated_generation_gwh_2015', 'estimated_generation_gwh_2016',
       'estimated_generation_gwh_2017', 'estimated_generation_note_2013',
       'estimated_generation_note_2014', 'estimated_generation_note_2015',
       'estimated_generation_note_2016', 'estimated_generation_note_2017'],
      dtype='object')

In [19]:
droplist3 = ['estimated_generation_note_2013', 'estimated_generation_note_2014',
	'estimated_generation_note_2015', 'estimated_generation_note_2016',
    'estimated_generation_note_2017','other_fuel1',
    'other_fuel2','other_fuel3', 'commissioning_year', 'gppd_idnr','owner',
    'source', 'url', 'geolocation_source', 'wepp_id', 'year_of_capacity_data',
    'generation_data_source','generation_gwh_2013', 'generation_gwh_2014', 
    'generation_gwh_2015', 'generation_gwh_2016', 'generation_gwh_2017', 
    'generation_gwh_2018', 'generation_gwh_2019','name']

plants1 = plants1.drop(droplist3, axis=1)
plants1.head(3)

,country,country_long,capacity_mw,latitude,longitude,primary_fuel,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017
0,AFG,Afghanistan,33.0,32.322,65.119,Hydro,123.77,162.90,97.39,137.76,119.50
1,AFG,Afghanistan,10.0,31.670,65.795,Solar,18.43,17.48,18.25,17.70,18.29
2,AFG,Afghanistan,10.0,31.623,65.792,Solar,18.64,17.58,19.10,17.62,18.72


In [20]:
plants1['primary_fuel'].unique()

array(['Hydro', 'Solar', 'Gas', 'Other', 'Oil', 'Wind', 'Nuclear', 'Coal',
       'Waste', 'Biomass', 'Wave and Tidal', 'Petcoke', 'Geothermal',
       'Storage', 'CHP'], dtype=object)

In [21]:
droplist = ['Other', 'Oil', 'Waste', 'Wave and Tidal','Petcoke']

plants1 = plants1[~plants1['primary_fuel'].isin(droplist)]

In [22]:
plants1['primary_fuel'].unique()

array(['Hydro', 'Solar', 'Gas', 'Wind', 'Nuclear', 'Coal', 'Biomass',
       'Geothermal', 'Storage', 'CHP'], dtype=object)

In [23]:
# save the dataframe to a CSV file
plants1.to_csv('Final/CleanData/globalplants1.csv', index=False)
#plants2.to_csv('Final/CleanData/globalplants2.csv', index=False)

### IMF prices

In [24]:
matprice.head()

,Component,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Chromium,8938.867278,8856.813487,8844.760673,8219.054096,8925.346912,10905.248396,9412.283885,6930.035686,9096.638853,10826.342759
1,Cobalt,27090.822506,30780.731957,28451.660699,25463.368543,55832.170981,72911.236898,33177.632305,31444.926902,51235.518380,63764.719304
2,Copper,7331.490765,6863.396581,5510.457113,4867.897430,6169.939942,6529.798371,6010.145430,6174.557917,9317.409166,8828.916814
3,Manganese,2317.528589,2236.811143,2012.081459,1776.361342,1977.000949,2378.426874,1899.755862,1713.635815,4053.592878,4501.483390
4,Nickel,15029.988580,16893.374548,11862.634519,9595.179081,10409.634946,13114.063889,13913.908693,13790.426574,18467.051874,25867.262713


In [25]:
# use melt function to reshape dataframe
matprice1 = pd.melt(matprice, id_vars=['Component'], var_name='Year', value_name='Price_per_Ton')
matprice1.head()

,Component,Year,Price_per_Ton
0,Chromium,2013,8938.867278
1,Cobalt,2013,27090.822506
2,Copper,2013,7331.490765
3,Manganese,2013,2317.528589
4,Nickel,2013,15029.988580


In [26]:
matprice1['Year'] = pd.to_datetime(matprice1['Year'], format='%Y')
matprice1.head()

,Component,Year,Price_per_Ton
0,Chromium,2013-01-01,8938.867278
1,Cobalt,2013-01-01,27090.822506
2,Copper,2013-01-01,7331.490765
3,Manganese,2013-01-01,2317.528589
4,Nickel,2013-01-01,15029.988580


In [27]:
matprice1.to_csv('Final/CleanData/Material_prices.csv', index=False)

### IEA Materials

In [28]:
matuse.head()

,Plant Type,Copper,Nickel,Manganese,Cobalt,Chromium,Zinc,Rare earths,Silicon,Others
0,Offshore wind,8000.0,240.00,790.00,0.00,525.0,5500.00,239.0,0.0,6.00
1,Onshore wind,2900.0,403.50,780.00,0.00,470.0,5500.00,14.0,0.0,0.00
2,Solar PV,2822.1,1.30,0.00,0.00,0.0,29.99,0.0,3948.3,31.95
3,Nuclear,1473.0,1297.40,147.69,0.00,2190.0,0.00,0.5,0.0,94.28
4,Coal,1150.0,721.04,4.63,201.46,307.5,0.00,0.0,0.0,33.90


In [29]:
matuse1 = matuse.drop(['Rare earths','Others'], axis=1)

matuse1 = pd.melt(matuse1, id_vars=['Plant Type'], var_name='Component', value_name='Kg/MW of Capacity')

matuse1.head()

,Plant Type,Component,Kg/MW of Capacity
0,Offshore wind,Copper,8000.0
1,Onshore wind,Copper,2900.0
2,Solar PV,Copper,2822.1
3,Nuclear,Copper,1473.0
4,Coal,Copper,1150.0


In [30]:
merge_comp = pd.merge(matuse1,matprice,on='Component',how='left')

merge_comp.head()

,Plant Type,Component,Kg/MW of Capacity,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Offshore wind,Copper,8000.0,7331.490765,6863.396581,5510.457113,4867.89743,6169.939942,6529.798371,6010.14543,6174.557917,9317.409166,8828.916814
1,Onshore wind,Copper,2900.0,7331.490765,6863.396581,5510.457113,4867.89743,6169.939942,6529.798371,6010.14543,6174.557917,9317.409166,8828.916814
2,Solar PV,Copper,2822.1,7331.490765,6863.396581,5510.457113,4867.89743,6169.939942,6529.798371,6010.14543,6174.557917,9317.409166,8828.916814
3,Nuclear,Copper,1473.0,7331.490765,6863.396581,5510.457113,4867.89743,6169.939942,6529.798371,6010.14543,6174.557917,9317.409166,8828.916814
4,Coal,Copper,1150.0,7331.490765,6863.396581,5510.457113,4867.89743,6169.939942,6529.798371,6010.14543,6174.557917,9317.409166,8828.916814


In [31]:
for n in range(2013,2023):
    merge_comp[f'{n}'] = merge_comp[f'{n}']/1000
    merge_comp[f'price in {n}'] = merge_comp[f'{n}'] * merge_comp['Kg/MW of Capacity']
    merge_comp = merge_comp.drop(f'{n}',axis=1)
merge_comp.head()

,Plant Type,Component,Kg/MW of Capacity,price in 2013,price in 2014,price in 2015,price in 2016,price in 2017,price in 2018,price in 2019,price in 2020,price in 2021,price in 2022
0,Offshore wind,Copper,8000.0,58651.926120,54907.172644,44083.656902,38943.179437,49359.519535,52238.386968,48081.163437,49396.463339,74539.273327,70631.334513
1,Onshore wind,Copper,2900.0,21261.323218,19903.850084,15980.325627,14116.902546,17892.825831,18936.415276,17429.421746,17906.217960,27020.486581,25603.858761
2,Solar PV,Copper,2822.1,20690.200088,19369.191490,15551.061018,13737.693336,17412.187510,18427.743983,16961.231417,17425.219898,26294.660407,24916.086141
3,Nuclear,Copper,1473.0,10799.285897,10109.783163,8116.903327,7170.412914,9088.321534,9618.393000,8852.944218,9095.123812,13724.543701,13004.994467
4,Coal,Copper,1150.0,8431.214380,7892.906068,6337.025680,5598.082044,7095.430933,7509.268127,6911.667244,7100.741605,10715.020541,10153.254336


In [32]:
merge_comp.to_csv('Final/CleanData/Plant_Materials.csv', index=False)